In [1]:
import keras
import scipy
import numpy as np
import pandas as pd
import tensorflow as tf
from keras import layers

import matplotlib.pyplot as plt
from scipy.stats import pearsonr
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

from numba import jit, float64, int64

plt.rc('font', size=12)          # controls default text sizes
plt.rc('axes', titlesize=12)     # fontsize of the axes title
plt.rc('axes', labelsize=15)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=12)    # fontsize of the tick labels
plt.rc('ytick', labelsize=12)    # fontsize of the tick labels
plt.rc('legend', fontsize=15)    # legend fontsize

In [2]:
def make_train_test(data, test_size=0.3):
    sbd = data[data['sbj'] == 0]
    X_train, X_test, y1, y2 = train_test_split(sbd, sbd, test_size=test_size, random_state=1111)
    for i in range(1, data.sbj.max()+1):
        sbd = data[data['sbj'] == i]
        tr, ts, y1, y2 = train_test_split(sbd, sbd, test_size=test_size, random_state=1111)
        X_train = pd.concat([X_train, tr])
        X_test = pd.concat([X_test, ts])
        
    return X_train.reset_index(drop=True), X_test.reset_index(drop=True)

In [3]:
@jit(nopython=True)
def Race(v, b, ndt=0, dt=0.001):
    x = np.array([0.0, 0.0])
    rt = 0
    ch = -1
    stop = False
    while not stop:
        x[0] += v[0]*dt + np.random.normal(0, 1)*np.sqrt(dt)
        x[1] += v[1]*dt + np.random.normal(0, 1)*np.sqrt(dt)
        rt += dt
        if x[0]>= b and x[1]<b:
            ch = 0.0
            stop = True
        elif x[1]>= b and x[0]<b:
            ch = 1.0
            stop = True
        elif x[0]>= b and x[1]>=b:
            rt = 0
            x = np.array([0.0, 0.0])
    return rt+ndt, ch

In [4]:
data_df = pd.read_csv('../../_00_data/Sepulveda2020_perceptual_prep.csv', index_col=0)

min_rt = data_df.groupby(by='sbj')['rt'].min()

In [5]:
tr_df, ts_df = make_train_test(data_df, test_size=0.4)

obs_train = np.asarray(tr_df[['rt', 'choice', 'sbj']])
X_train = np.asarray(tr_df[['item_value_0', 'item_value_1', 'gaze_0', 'gaze_1', 'goal']])

obs_test = np.asarray(ts_df[['rt', 'choice', 'sbj']])
X_test = np.asarray(ts_df[['item_value_0', 'item_value_1', 'gaze_0', 'gaze_1', 'goal']])

print(X_train.shape, X_test.shape)

(4521, 5) (3034, 5)


In [6]:
def fpt(t, b, v):
    return tf.cast(tf.math.greater(t, 0), tf.float32)*(b/(2*np.pi*t**3)**(1/2) * keras.backend.exp(-1/(2*t) * (v*t-b)**2)) + tf.cast(tf.math.less_equal(t, 0), tf.float32)*0.1**10

def phi(x):
    return 0.5*tf.math.erf(x/tf.sqrt(2.0)) + 0.5

def cdf(t, b, v):
    return tf.cast(tf.math.greater(t, 0), tf.float32)*(phi((v*t-b)/tf.sqrt(t)) + tf.exp(2*v*b)*phi((-v*t-b)/tf.sqrt(t))) + (tf.cast(tf.math.less_equal(t, 0), tf.float32)*0)

def G2(data, pred):

    sbj_indices = tf.cast(data[:, 2], tf.int32)

    # Gather values directly using TensorFlow operations
    gathered_min_rt = tf.gather(min_rt, sbj_indices)
    sigmoid_weights = tf.gather(model.layers[-1].weights[-1], sbj_indices)
    ndt = gathered_min_rt * tf.math.sigmoid(sigmoid_weights)

    thr_weights = tf.gather(model.layers[-1].weights[-2], sbj_indices)
    thr = tf.math.log(1 + tf.exp(thr_weights))

    scl_weights = tf.gather(model.layers[-1].weights[-3], sbj_indices)
    scl = tf.math.log(1 + tf.exp(scl_weights))

    ll1 = (1 - data[:, 1]) * fpt(data[:, 0]-ndt, thr, scl*pred[:, 0]) * (1.0 - cdf(data[:, 0]-ndt, thr, scl*pred[:, 1]))
    ll2 = (data[:, 1]) * fpt(data[:, 0]-ndt, thr, scl*pred[:, 1]) * (1.0 - cdf(data[:, 0]-ndt, thr, scl*pred[:, 0]))

    return -2*tf.reduce_sum(tf.math.log(tf.maximum(ll1 + ll2, 1e-14)), axis=-1)

In [7]:
class Out_layer(keras.layers.Layer):
    def __init__(self, units=2, input_dim=32, n_sbj_prms=10):
        super().__init__()
        self.w = self.add_weight(shape=(input_dim, units), initializer="random_normal", trainable=True)
        self.b = self.add_weight(shape=(units,), initializer="uniform", trainable=True)
        
        self.scaling = self.add_weight(shape=(n_sbj_prms,), initializer="uniform", trainable=True)
        self.threshold = self.add_weight(shape=(n_sbj_prms,), initializer="uniform", trainable=True)
        self.non_decision_time = self.add_weight(shape=(n_sbj_prms,), initializer="uniform", trainable=True)

    def call(self, inputs):
        return tf.math.log(1 + tf.exp(tf.matmul(inputs, self.w) + self.b))
    
def make_model(layers_neuron, input_shape, output_shape, activation, n_sbj_prms):
    keras.backend.clear_session()

    model = keras.Sequential()
    model.add(layers.Input(shape=(input_shape,)))
    
    for l in layers_neuron:
        model.add(layers.Dense(l, activation=activation))
    
    model.add(Out_layer(units=output_shape, input_dim=layers_neuron[-1], n_sbj_prms=n_sbj_prms))

    print(model.summary())
    
    return model

In [8]:
layers_neuron = [10, 10, 10, 8]
input_shape = 5
output_shape = 2
activation = 'tanh'
n_sbj = tr_df.sbj.nunique()

model = make_model(layers_neuron, input_shape, output_shape, activation, n_sbj)

model.build()
model.compile(optimizer="RMSprop", loss=G2)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 10)                60        
                                                                 
 dense_1 (Dense)             (None, 10)                110       
                                                                 
 dense_2 (Dense)             (None, 10)                110       
                                                                 
 dense_3 (Dense)             (None, 8)                 88        
                                                                 
 out_layer (Out_layer)       (None, 2)                 114       
                                                                 
Total params: 482 (1.88 KB)
Trainable params: 482 (1.88 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [9]:
sbj_indices = tf.cast(obs_train[:, 2], tf.int32)
obs_train = tf.cast(obs_train, tf.float32)
X_train = tf.convert_to_tensor(X_train)
X_train = tf.cast(X_train, tf.float32)

min_rt = tf.convert_to_tensor(min_rt)
min_rt = tf.cast(min_rt, tf.float32)

In [10]:
history = model.fit(X_train, 
                    obs_train, 
                    shuffle=False, 
                    batch_size=X_train.shape[0], 
                    epochs=3500)

Epoch 1/3500
1/1 [==============================] - 1s 580ms/step - loss: 42589.6016
Epoch 2/3500
1/1 [==============================] - 0s 3ms/step - loss: 42258.4375
Epoch 3/3500
1/1 [==============================] - 0s 2ms/step - loss: 42027.5430
Epoch 4/3500
1/1 [==============================] - 0s 3ms/step - loss: 41839.0859
Epoch 5/3500
1/1 [==============================] - 0s 3ms/step - loss: 41675.0000
Epoch 6/3500
1/1 [==============================] - 0s 3ms/step - loss: 41527.0625
Epoch 7/3500
1/1 [==============================] - 0s 3ms/step - loss: 41390.7305
Epoch 8/3500
1/1 [==============================] - 0s 3ms/step - loss: 41263.1484
Epoch 9/3500
1/1 [==============================] - 0s 3ms/step - loss: 41142.3359
Epoch 10/3500
1/1 [==============================] - 0s 3ms/step - loss: 41026.8438
Epoch 11/3500
1/1 [==============================] - 0s 3ms/step - loss: 40915.5859
Epoch 12/3500
1/1 [==============================] - 0s 3ms/step - loss: 40807.7266

1/1 [==============================] - 0s 3ms/step - loss: 32676.4062
Epoch 195/3500
1/1 [==============================] - 0s 3ms/step - loss: 32656.4844
Epoch 196/3500
1/1 [==============================] - 0s 3ms/step - loss: 32636.6738
Epoch 197/3500
1/1 [==============================] - 0s 3ms/step - loss: 32616.9609
Epoch 198/3500
1/1 [==============================] - 0s 3ms/step - loss: 32597.3477
Epoch 199/3500
1/1 [==============================] - 0s 3ms/step - loss: 32577.8262
Epoch 200/3500
1/1 [==============================] - 0s 3ms/step - loss: 32558.4102
Epoch 201/3500
1/1 [==============================] - 0s 3ms/step - loss: 32539.0918
Epoch 202/3500
1/1 [==============================] - 0s 3ms/step - loss: 32519.8594
Epoch 203/3500
1/1 [==============================] - 0s 3ms/step - loss: 32500.7285
Epoch 204/3500
1/1 [==============================] - 0s 2ms/step - loss: 32481.6855
Epoch 205/3500
1/1 [==============================] - 0s 3ms/step - loss: 32462.

1/1 [==============================] - 0s 3ms/step - loss: 29894.0527
Epoch 387/3500
1/1 [==============================] - 0s 3ms/step - loss: 29882.5742
Epoch 388/3500
1/1 [==============================] - 0s 3ms/step - loss: 29871.1250
Epoch 389/3500
1/1 [==============================] - 0s 3ms/step - loss: 29859.6914
Epoch 390/3500
1/1 [==============================] - 0s 3ms/step - loss: 29848.2715
Epoch 391/3500
1/1 [==============================] - 0s 3ms/step - loss: 29836.8711
Epoch 392/3500
1/1 [==============================] - 0s 3ms/step - loss: 29825.4902
Epoch 393/3500
1/1 [==============================] - 0s 3ms/step - loss: 29814.1230
Epoch 394/3500
1/1 [==============================] - 0s 3ms/step - loss: 29802.7773
Epoch 395/3500
1/1 [==============================] - 0s 3ms/step - loss: 29791.4453
Epoch 396/3500
1/1 [==============================] - 0s 3ms/step - loss: 29780.1328
Epoch 397/3500
1/1 [==============================] - 0s 3ms/step - loss: 29768.

1/1 [==============================] - 0s 3ms/step - loss: 27948.7969
Epoch 579/3500
1/1 [==============================] - 0s 3ms/step - loss: 27939.8203
Epoch 580/3500
1/1 [==============================] - 0s 3ms/step - loss: 27930.8477
Epoch 581/3500
1/1 [==============================] - 0s 3ms/step - loss: 27921.8867
Epoch 582/3500
1/1 [==============================] - 0s 3ms/step - loss: 27912.9414
Epoch 583/3500
1/1 [==============================] - 0s 3ms/step - loss: 27904.0020
Epoch 584/3500
1/1 [==============================] - 0s 3ms/step - loss: 27895.0742
Epoch 585/3500
1/1 [==============================] - 0s 3ms/step - loss: 27886.1602
Epoch 586/3500
1/1 [==============================] - 0s 3ms/step - loss: 27877.2539
Epoch 587/3500
1/1 [==============================] - 0s 3ms/step - loss: 27868.3613
Epoch 588/3500
1/1 [==============================] - 0s 3ms/step - loss: 27859.4785
Epoch 589/3500
1/1 [==============================] - 0s 3ms/step - loss: 27850.

1/1 [==============================] - 0s 4ms/step - loss: 26399.2461
Epoch 771/3500
1/1 [==============================] - 0s 3ms/step - loss: 26391.9355
Epoch 772/3500
1/1 [==============================] - 0s 3ms/step - loss: 26384.6328
Epoch 773/3500
1/1 [==============================] - 0s 3ms/step - loss: 26377.3359
Epoch 774/3500
1/1 [==============================] - 0s 3ms/step - loss: 26370.0430
Epoch 775/3500
1/1 [==============================] - 0s 3ms/step - loss: 26362.7539
Epoch 776/3500
1/1 [==============================] - 0s 3ms/step - loss: 26355.4727
Epoch 777/3500
1/1 [==============================] - 0s 3ms/step - loss: 26348.1953
Epoch 778/3500
1/1 [==============================] - 0s 3ms/step - loss: 26340.9238
Epoch 779/3500
1/1 [==============================] - 0s 3ms/step - loss: 26333.6582
Epoch 780/3500
1/1 [==============================] - 0s 3ms/step - loss: 26326.3945
Epoch 781/3500
1/1 [==============================] - 0s 3ms/step - loss: 26319.

...

1/1 [==============================] - 0s 2ms/step - loss: 21483.1523
Epoch 2673/3500
1/1 [==============================] - 0s 2ms/step - loss: 21481.3086
Epoch 2674/3500
1/1 [==============================] - 0s 2ms/step - loss: 21480.2031
Epoch 2675/3500
1/1 [==============================] - 0s 2ms/step - loss: 21478.3340
Epoch 2676/3500
1/1 [==============================] - 0s 2ms/step - loss: 21477.2422
Epoch 2677/3500
1/1 [==============================] - 0s 2ms/step - loss: 21475.3730
Epoch 2678/3500
1/1 [==============================] - 0s 2ms/step - loss: 21474.2852
Epoch 2679/3500
1/1 [==============================] - 0s 2ms/step - loss: 21472.4551
Epoch 2680/3500
1/1 [==============================] - 0s 2ms/step - loss: 21471.3809
Epoch 2681/3500
1/1 [==============================] - 0s 2ms/step - loss: 21469.4590
Epoch 2682/3500
1/1 [==============================] - 0s 2ms/step - loss: 21468.3672
Epoch 2683/3500
1/1 [==============================] - 0s 2ms/step - l

1/1 [==============================] - 0s 2ms/step - loss: 21038.5273
Epoch 2863/3500
1/1 [==============================] - 0s 2ms/step - loss: 21034.8398
Epoch 2864/3500
1/1 [==============================] - 0s 2ms/step - loss: 21033.2266
Epoch 2865/3500
1/1 [==============================] - 0s 3ms/step - loss: 21029.4570
Epoch 2866/3500
1/1 [==============================] - 0s 2ms/step - loss: 21027.9688
Epoch 2867/3500
1/1 [==============================] - 0s 2ms/step - loss: 21023.3047
Epoch 2868/3500
1/1 [==============================] - 0s 2ms/step - loss: 21020.8086
Epoch 2869/3500
1/1 [==============================] - 0s 2ms/step - loss: 21016.4004
Epoch 2870/3500
1/1 [==============================] - 0s 2ms/step - loss: 21014.0391
Epoch 2871/3500
1/1 [==============================] - 0s 2ms/step - loss: 21010.1133
Epoch 2872/3500
1/1 [==============================] - 0s 2ms/step - loss: 21008.1523
Epoch 2873/3500
1/1 [==============================] - 0s 2ms/step - l

1/1 [==============================] - 0s 2ms/step - loss: 20570.3008
Epoch 3053/3500
1/1 [==============================] - 0s 3ms/step - loss: 20566.6309
Epoch 3054/3500
1/1 [==============================] - 0s 7ms/step - loss: 20567.3145
Epoch 3055/3500
1/1 [==============================] - 0s 3ms/step - loss: 20569.1875
Epoch 3056/3500
1/1 [==============================] - 0s 3ms/step - loss: 20583.4551
Epoch 3057/3500
1/1 [==============================] - 0s 3ms/step - loss: 20590.0000
Epoch 3058/3500
1/1 [==============================] - 0s 3ms/step - loss: 20578.7500
Epoch 3059/3500
1/1 [==============================] - 0s 2ms/step - loss: 20565.7402
Epoch 3060/3500
1/1 [==============================] - 0s 2ms/step - loss: 20565.8828
Epoch 3061/3500
1/1 [==============================] - 0s 2ms/step - loss: 20560.0723
Epoch 3062/3500
1/1 [==============================] - 0s 2ms/step - loss: 20562.2188
Epoch 3063/3500
1/1 [==============================] - 0s 2ms/step - l

1/1 [==============================] - 0s 3ms/step - loss: 20232.1562
Epoch 3243/3500
1/1 [==============================] - 0s 2ms/step - loss: 20207.5547
Epoch 3244/3500
1/1 [==============================] - 0s 2ms/step - loss: 20204.7266
Epoch 3245/3500
1/1 [==============================] - 0s 2ms/step - loss: 20194.0156
Epoch 3246/3500
1/1 [==============================] - 0s 2ms/step - loss: 20185.2266
Epoch 3247/3500
1/1 [==============================] - 0s 2ms/step - loss: 20179.5547
Epoch 3248/3500
1/1 [==============================] - 0s 2ms/step - loss: 20173.8086
Epoch 3249/3500
1/1 [==============================] - 0s 2ms/step - loss: 20170.2031
Epoch 3250/3500
1/1 [==============================] - 0s 2ms/step - loss: 20168.0430
Epoch 3251/3500
1/1 [==============================] - 0s 2ms/step - loss: 20167.2031
Epoch 3252/3500
1/1 [==============================] - 0s 2ms/step - loss: 20167.3633
Epoch 3253/3500
1/1 [==============================] - 0s 2ms/step - l

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [11]:
pred_v = model.predict(X_test)
threshold = np.asanyarray(tf.math.log(1 + tf.exp(model.layers[-1].weights[-2])))
ndt = np.asanyarray(np.asanyarray(min_rt)*tf.math.sigmoid(model.layers[-1].weights[-1]))

95/95 [==============================] - 0s 360us/step


In [12]:
pred_bhv = {'sbj': [],
            'rt': [],
            'ch': [],
            'goal':[],
            'value_diff': [],
            'gaze_diff': []}

scaling = np.asanyarray(tf.math.log(1 + tf.exp(model.layers[-1].weights[-3])))
scaling = np.asanyarray([scaling[i] for i in ts_df['sbj']])

for n in range(20):
    for s in range(ts_df.shape[0]):
        rt, ch = Race(scaling[s]*pred_v[s,:], threshold[ts_df.sbj[s]], ndt[ts_df.sbj[s]])
        pred_bhv['sbj'].append(ts_df.sbj[s])
        pred_bhv['rt'].append(rt)
        pred_bhv['ch'].append(ch)
        pred_bhv['goal'].append(ts_df.goal[s])
        pred_bhv['value_diff'].append(ts_df['Right Value - Left Value'][s])
        pred_bhv['gaze_diff'].append(ts_df['Dwell Time Advantage (R - L)'][s])
        
pred_bhv = pd.DataFrame(pred_bhv)

In [13]:
pred_bhv.to_csv('Sp2020_preceptual_out_of_sample.csv')
ts_df.to_csv('Sp2020_preceptual_test.csv')